<a href="https://colab.research.google.com/github/HsunhuiLin/CodeGenerationModel_sql_Assistant/blob/main/NLP_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Translating natural language instructions to SQL


In [1]:
!pip install sqlalchemy
# CSV --> SQL RAM (in-memory database) --> query using SQL -> results

In [2]:
import os
os.getcwd()
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/composed-setup-416711-624ff8c42fc5.json'

In [3]:
import pandas as pd
from sqlalchemy import create_engine, text

In [5]:
df=pd.read_csv('/content/penguins.csv')

database = create_engine('sqlite:///:memory:', echo=False)
table = df.to_sql(name='Penguins', con=database)

In [ ]:
with database.connect() as conn:
  result = conn.execute(text("SELECT * FROM Penguins"))

# check results of all rows
result.all()

In [7]:
# Inspect dataset properties and feed it to model's prefix
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [8]:
def create_prefix(query):
  prefix = f'''Return SQL statement that answers the following query:
  {query}

  Properties of the table:
  #   Column             Non-Null Count  Dtype
  ---  ------             --------------  -----
  0   species            344 non-null    object
  1   island             344 non-null    object
  2   bill_length_mm     342 non-null    float64
  3   bill_depth_mm      342 non-null    float64
  4   flipper_length_mm  342 non-null    float64
  5   body_mass_g        342 non-null    float64
  6   sex                333 non-null    object

  Example rows:
  (267, 'Gentoo', 'Biscoe', 50.5, 15.9, 225.0, 5400.0, 'MALE'),
  (268, 'Gentoo', 'Biscoe', 44.9, 13.3, 213.0, 5100.0, 'FEMALE'),
  (269, 'Gentoo', 'Biscoe', 45.2, 15.8, 215.0, 5300.0, 'MALE'),
  (270, 'Gentoo', 'Biscoe', 46.6, 14.2, 210.0, 4850.0, 'FEMALE'),
  (271, 'Gentoo', 'Biscoe', 48.5, 14.1, 220.0, 5300.0, 'MALE'),

  Return only the sql statement.
  '''

  return prefix

In [17]:
def user_input():
  query = input("What information you need from the Penguin table?\n")
  return create_prefix(query)

In [12]:
from vertexai.language_models import CodeGenerationModel
code_gen_model = CodeGenerationModel.from_pretrained('code-bison')

In [13]:
# To clean the model-generated output to a string of SQL statement
def clean_text(statement):
  return statement.replace("```sql",'').replace("```",'').replace("\n", " ")

In [18]:
def nlp_assistant():
  print("Hi! I am your technical assistant. You can ask me anything about the database we have.")
  print('\n\n')
  prefix = user_input()
  statement = code_gen_model.predict(prefix=prefix)
  clean_sql = clean_text(statement.text)

  with database.connect() as conn:
    result = conn.execute(text(clean_sql))

  print("\n===================================")
  print("Here is your result:\n")
  print(result.all())


In [19]:
nlp_assistant()

Hi! I am your technical assistant. You can ask me anything about the database we have.



What information you need from the Penguin table?
How many male are there in each species?

Here is your result:

[('Adelie', 73), ('Chinstrap', 34), ('Gentoo', 61)]
